In [1]:
import os
import shutil
import json
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

In [2]:
def process_qdess_results(fem_dict):
    
    from re import M
    import pandas as pd
    # Initialize a dictionary to store aggregated results
    aggregated_data = {}

    # Map for side
    side_map = {
        'ant_fem': 'AN',
        'med_wb': 'MC',
        'lat_wb': 'LC',
        'med_post': 'MP',
        'lat_post': 'LP'
    }

    # Prepare a function to update the aggregated dictionary
    def update_aggregated_data(data, layer, side, measurement, value):
        key = (layer, side)
        if key not in data:
            data[key] = {"T2Mean": None, "T2Median": None, "T2Std": None, "ThicknessMean": None, "ThicknessMedian": None, "ThicknessStd": None}
        data[key][measurement] = value

    # Process the dictionary to fill the aggregated data dictionary
    for key, value in fem_dict.items():
        parts = key.split('_')
        if len(parts) >= 5:
            region = '_'.join(parts[:2])
            measure_type = parts[-1]
            
            # Determine Layer
            if 'deep' in key:
                layer = 'Deep'
            elif 'superficial' in key:
                layer = 'Superficial'
            else:
                layer = 'Overall'

            side = side_map.get(region, region)

            # Determine if it's T2 or not
            if measure_type == 'mean':
                if 't2' in key:
                    update_aggregated_data(aggregated_data, layer, side, 'T2 Mean', value)
                elif 'mm' in key:
                    update_aggregated_data(aggregated_data, layer, side, 'Thickness Mean', value)
            elif measure_type == 'median':
                if 't2' in key:
                    update_aggregated_data(aggregated_data, layer, side, 'T2 Median', value)
                elif 'mm' in key:
                    update_aggregated_data(aggregated_data, layer, side, 'Thickness Median', value)
            elif measure_type == 'std':
                if 't2' in key:
                    update_aggregated_data(aggregated_data, layer, side, 'T2 Std', value)
                elif 'mm' in key:
                    update_aggregated_data(aggregated_data, layer, side, 'Thickness Std', value)

    # Convert the aggregated dictionary to a DataFrame
    df = pd.DataFrame([
        {
            'Layer': layer,
            'Sub-region': side,
            'T2Mean': measurements['T2 Mean'],
            'T2Median': measurements['T2 Median'],
            'T2Std': measurements['T2 Std'],
            'ThicknessMean': measurements.get('Thickness Mean'),
            'ThicknessMedian': measurements.get('Thickness Median'),
            'ThicknessStd': measurements.get('Thickness Std')
        }
        for (layer, side), measurements in aggregated_data.items()
    ])

    # Reorder columns
    df = df[['Layer', 'Sub-region', 'T2Mean', 'T2Median', 'T2Std', 'ThicknessMean', 'ThicknessMedian', 'ThicknessStd']]

    return df

In [3]:
dir_path= Path('/dataNAS/people/anoopai/DESS_ACL_study')
data_path= dir_path / 'data/'
nsm_results_path= dir_path/ 'results/Bscore_and_thickness'

# List to keep track of the dictionaries
analysis_complete = []

qdess_df_all= pd.DataFrame()
nsm_df_all= pd.DataFrame()

# Find data folder and results folder
for root, dirs, files in os.walk(data_path):
    for dir in dirs:
        sub_dir_path= Path(root) / dir
        if 'results' in sub_dir_path.name \
            and '42-C' not in str(sub_dir_path.parent) \
                and 'VISIT-6' not in str(sub_dir_path.parent) \
                    and ('27-P/VISIT-5/clat' not in str(sub_dir_path.parent)):   
            main_dir_path = sub_dir_path.parent
            results_path = Path(main_dir_path) / 'results'
            qdess_results_file = Path(results_path) / 'qdess_results.json' 
            nsm_recon_file = Path(results_path) / 'NSM_recon_params.json'
            
            # print('Processing:', results_path)
            # Split the path into components
            sub_component = results_path.parts[6]  # '11-P'
            visit_component = results_path.parts[7]  # 'VISIT-1'
            knee_component = results_path.parts[8]  # 'clat'
            
            if os.path.exists(qdess_results_file):
                # print('Processing:', results_path)
            
                with open(qdess_results_file) as f:
                    qdess_results = json.load(f)
                    
                    # Filter dictionary to include only keys that contain 'fem'
                    fem_dict = {key: value for key, value in qdess_results.items() if 'fem' in key}
                    
                    qdess_df= process_qdess_results(fem_dict)
                    qdess_df['Subject']= sub_component
                    qdess_df['Visit']= visit_component[-1]
                    qdess_df['Knee']= knee_component
                    
                    cols= qdess_df.columns
                    cols_list= cols.to_list()
                    unique_cols = [col for col in cols_list if col not in ['Subject', 'Visit', 'Knee']]
                    cols_order = ['Subject', 'Visit', 'Knee'] + unique_cols
                    qdess_df= qdess_df[cols_order]
                    
                    qdess_df_all= pd.concat([qdess_df_all, qdess_df])
                                    
            else:
                print('Results not found:', results_path)
            
# print('QDESS Results')
print(qdess_df_all)

qdess_df_all.to_excel(nsm_results_path / 'data_files/qdess_results.xlsx', index=False)

Results not found: /dataNAS/people/anoopai/DESS_ACL_study/data/27-P/VISIT-5/aclr/results
   Subject Visit  Knee        Layer Sub-region     T2Mean  T2Median  \
0     11-P     2  aclr      Overall         AN  28.504986      27.3   
1     11-P     2  aclr      Overall         MC  27.556913      26.2   
2     11-P     2  aclr      Overall         LC  30.330831      27.2   
3     11-P     2  aclr      Overall         MP  30.431823      29.5   
4     11-P     2  aclr      Overall         LP  29.978966      28.4   
..     ...   ...   ...          ...        ...        ...       ...   
10    35-C     5  ctrl  Superficial         LC  36.766469      35.6   
11    35-C     5  ctrl         Deep         MP  24.628774      24.0   
12    35-C     5  ctrl  Superficial         MP  31.199952      30.1   
13    35-C     5  ctrl         Deep         LP  21.026027      20.6   
14    35-C     5  ctrl  Superficial         LP  28.725357      27.2   

        T2Std  ThicknessMean  ThicknessMedian  ThicknessSt

In [4]:
dir_path= Path('/dataNAS/people/anoopai/DESS_ACL_study')
data_path= dir_path / 'data/'
nsm_results_path= dir_path/ 'results/Bscore_and_thickness'

# List to keep track of the dictionaries
analysis_complete = []

qdess_df_all= pd.DataFrame()
nsm_df_all= pd.DataFrame()

# Find data folder and results folder
for root, dirs, files in os.walk(data_path):
    for dir in dirs:
        sub_dir_path= Path(root) / dir
        if 'results' in sub_dir_path.name \
            and '42-C' not in str(sub_dir_path.parent) \
                and 'VISIT-6' not in str(sub_dir_path.parent):  
            main_dir_path = sub_dir_path.parent
            results_path = Path(main_dir_path) / 'results'
            qdess_results_file = Path(results_path) / 'qdess_results.json' 
            nsm_results_file = Path(results_path) / 'NSM_recon_params.json'
            
            
            # print('Processing:', results_path)
            # Split the path into components
            sub_component = results_path.parts[6]  # '11-P'
            visit_component = results_path.parts[7]  # 'VISIT-1'
            knee_component = results_path.parts[8]  # 'clat'
            
            if os.path.exists(nsm_results_file):
                
                with open(nsm_results_file) as f:
                    nsm_recon = json.load(f)
    
                # Collect the extracted information in a dictionary
                nsm_results = {
                    'Subject': sub_component,
                    'Visit': visit_component[-1],
                    'Knee': knee_component,
                    'Bscore': nsm_recon['Bscore'],
                    'scale': nsm_recon['scale'],
                    'assd_bone_mm': nsm_recon['assd_bone_mm'],
                    'assd_cartilage_mm': nsm_recon['assd_cartilage_mm'],
                    
                }
                
                nsm_df= pd.DataFrame(nsm_results, index=[0]).reset_index(drop=True)
                nsm_df_all= pd.concat([nsm_df_all, nsm_df])
                
            else:
                print('Results not found:', results_path)
            
# print('NSM Results')
print(nsm_df_all)

nsm_df_all.to_excel(nsm_results_path / 'data_files/nsm_results.xlsx', index=False)

   Subject Visit  Knee    Bscore     scale  assd_bone_mm  assd_cartilage_mm
0     11-P     2  aclr  1.213755  0.956195      0.242016           0.136475
0     11-P     2  clat  1.768894  0.971741      0.283746           0.143941
0     11-P     5  aclr  1.269275  0.961057      0.231375           0.146125
0     11-P     5  clat  1.581905  0.965180      0.288937           0.131319
0     11-P     1  aclr  1.286266  0.959444      0.233757           0.165380
..     ...   ...   ...       ...       ...           ...                ...
0     14-C     1  ctrl  0.752332  0.945463      0.229800           0.126727
0     35-C     3  ctrl  0.977927  0.975928      0.231336           0.155260
0     35-C     1  ctrl  1.023927  0.974444      0.242190           0.146634
0     35-C     2  ctrl  1.032462  0.979290      0.221899           0.151532
0     35-C     5  ctrl  1.002978  0.975705      0.257149           0.148585

[224 rows x 7 columns]
